In [248]:
from dotenv import load_dotenv
import os
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec
import json
load_dotenv()

True

In [247]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
pc.create_index(
    name='stock-advise',
    dimension=1536,
    metric='cosine',
    spec=ServerlessSpec(cloud='aws', region='us-east-1')
)



In [323]:
data = json.load(open("/Users/ceo/Desktop/headstarter_SWE/Final_Project/Buddyfin-AI/src/data/reviews.json"))

In [324]:
new_data = []
client = OpenAI()

for stocks in data['best_stocks']:
    input_text = f"{stocks['company']} {stocks['ticker']}"
    response = client.embeddings.create(
        input = input_text,
        model = 'text-embedding-3-small',
    )

    embedding = response.data[0].embedding
    new_data.append({
        "values": embedding,
        'id': stocks['company'],
        'metadata':{
            'ticker': stocks['ticker'],
            'analysis': stocks['analysts_recommendation'],
            'ratings': stocks['ratings'],
        }
        }
    )

for stocks in data['poor_stocks']:
    input_text = f"{stocks['company']} {stocks['ticker']}"
    response = client.embeddings.create(
        input = input_text,
        model = 'text-embedding-3-small',
    )

    embedding = response.data[0].embedding
    new_data.append({
        "values": embedding,
        'id': stocks['company'],
        'metadata':{
            'ticker': stocks['ticker'],
            'analysis': stocks['analysts_recommendation'],
            'ratings': stocks['ratings'],
        }
        }
    )



In [326]:
index = pc.Index('stock-advise')
index.upsert(
    vectors=new_data,
    namespace='ns1'
)

{'upserted_count': 15}